<a href="https://colab.research.google.com/github/Siftee150/How_Many_topics/blob/master/Copy_of_LDA%26SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 
import pandas as pd
import re
import nltk
import matplotlib
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
from wordcloud import WordCloud
import gensim
from gensim.models import CoherenceModel, LdaModel
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
rx_dict = {
    'keys': re.compile(r'Key: (?P<keys>.*)\n'),
    'medline': re.compile(r'Medline: (?P<medline>\d+)\n'),
    'authors': re.compile(r'Authors: (?P<authors>.*)\n'),
    'title': re.compile(r'Title: (?P<title>.*)'),
    'citation': re.compile(r'Citation: (?P<citation>.*)'),
    'type': re.compile(r'Type: (?P<type>.*)'),
    'genes': re.compile(r'Genes: (?P<genes>.*)'),
    'abstract': re.compile(r'Abstract: (?P<abstract>.*)'),
    'end':re.compile(r'-------------------')
    }

In [ ]:
def _findregex(line):
  for key,rx in rx_dict.items():
    match=rx.search(line)
    if match:
      return key,match
  return None,None

In [ ]:
def parse_file(filepath):
  data=[]
  keyval='-'
  medval='-'
  author_val='-'
  title_val='-'
  citation_val='-'
  type_val='-'
  genes_val='-'
  abstract_val='-'

  with open(filepath,'r') as file_object:
    line=file_object.readline()
    while line:
      key,match=_findregex(line)
      if key =='keys':
        keyval=match.group('keys')
      if key =='medline':
        medval=match.group('medline')
      if key =='authors':
        author_val=match.group('authors')
      if key =='title':
        title_val=match.group('title')
      if key =='citation':
        citation_val=match.group('citation')
      if key =='type':
        type_val=match.group('type')
      if key =='genes':
        genes_val=match.group('genes')
      if key =='abstract':
        abstract_val=match.group('abstract')
      if key=='end':
        row = {
          'Key': keyval,
          'Medline': medval,
          'Authors': author_val,
          'Title': title_val,
          'Citation': citation_val,
          'Type': type_val,
          'Genes': genes_val,
          'Abstract': abstract_val ,
          }
        data.append(row)
      line=file_object.readline()
    data=pd.DataFrame(data)
    return data

In [ ]:
Finaldata=parse_file('/content/textfile.txt')
print(Finaldata.head(1))

In [ ]:
 
def remove_punct(text):
  return text.translate(str.maketrans('', '', string.punctuation))
def remove_stopword(text):
  STOPWORD = set(stopwords.words('english'))
  return " ".join([word for word in str(text).split() if word not in STOPWORD])
lemmatise = WordNetLemmatizer()
def lemmatize_word(text):
    return " ".join([lemmatise.lemmatize(word) for word in text.split()])
ps = PorterStemmer()
def stem_doc(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [ ]:
Finaldata["Title_cleaned"]=Finaldata["Title"].str.lower()
Finaldata["Title_cleaned"]=Finaldata["Title_cleaned"].apply(lambda text:remove_punct(text))
Finaldata["Title_cleaned"]=Finaldata["Title_cleaned"].apply(lambda text:remove_stopword(text))
Finaldata["Title_cleaned"]=Finaldata["Title_cleaned"].apply(lambda text:lemmatize_word(text))
Finaldata["Title_cleaned"]=Finaldata["Title_cleaned"].apply(lambda text:stem_doc(text))
Finaldata.head()

In [ ]:
Finaldata=Finaldata.drop(['Citation', 'Genes', 'Type'],axis=1)

In [ ]:
Finaldata["Abstract_cleaned"]=Finaldata["Abstract"].str.lower()
Finaldata["Abstract_cleaned"]=Finaldata["Abstract_cleaned"].apply(lambda text:remove_punct(text))
Finaldata["Abstract_cleaned"]=Finaldata["Abstract_cleaned"].apply(lambda text:remove_stopword(text))
Finaldata["Abstract_cleaned"]=Finaldata["Abstract_cleaned"].apply(lambda text:lemmatize_word(text))
Finaldata["Abstract_cleaned"]=Finaldata["Abstract_cleaned"].apply(lambda text:stem_doc(text))
Finaldata.head()

In [ ]:
Finaldata["FinalText"]=Finaldata["Title_cleaned"]+" "+Finaldata["Abstract_cleaned"]
Finaldata["FinalText"]=Finaldata["FinalText"].apply(lambda text:nltk.word_tokenize(text))
Finaldata["FinalText"]

In [ ]:
Finaldata.head()

In [ ]:
total_word_count={}
for row in Finaldata["FinalText"]:
  for word in row:
    if word not in total_word_count.keys():
      total_word_count[word]=1
    else:
      total_word_count[word]+=1


In [ ]:
total_word_count
#max(total_word_count,key=total_word_count.get)

In [ ]:
wordcloud=WordCloud(width=800,height=800,background_color="white",min_font_size=10).generate_from_frequencies(total_word_count)
plt.figure(figsize = (8, 8)) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show() 

In [ ]:
lists=list(total_word_count.items())[:100]
#keys=total_word_count.keys()
#values=total_word_count.values()
plt.rcParams['figure.figsize']=(20,10)
for i in range(len(lists)):
  plt.bar(lists[i][0],lists[i][1])
plt.ylim(0,3000)
plt.show()

In [ ]:
Finaldata.to_csv('final_text.csv',index=False,header=False,
                 columns=['Key','Authors','FinalText'])

In [ ]:
#using lda algorithm on preprocessed data
dictionary=gensim.corpora.Dictionary(Finaldata["FinalText"])
#dictionary
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in Finaldata["FinalText"]]
bow_corpus

In [ ]:
bow_doc_4972 = bow_corpus[4972]
for i in range(len(bow_doc_4972)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4972[i][0],dictionary[bow_doc_4972[i][0]],bow_doc_4972[i][1]))

In [ ]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 10, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
for index, score in sorted(lda_model[bow_corpus[4972]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index)))

In [ ]:
bow_corpus[4972]

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus)) 
coherence_model_lda = CoherenceModel(model=lda_model, texts=Finaldata["FinalText"], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
def compute_coherence_value(dictionary, corpus, texts, start,limit,step):
    coherence_value = []
    model_list = []
    for total_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, num_topics=total_topics,id2word=dictionary,passes=10)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_value.append(coherencemodel.get_coherence())

    return model_list, coherence_value

In [ ]:
model_list, coherence_values = compute_coherence_value(dictionary=dictionary, corpus=bow_corpus, texts=Finaldata["FinalText"], start=1, limit=101, step=5)
coherence_values

In [ ]:
limit=101; start=1; step=5;
x = range(start, limit, step)
plt.figure(figsize=(12,12))
plt.plot(x, coherence_values)
plt.xlabel("Net number of Topics")
plt.ylabel("Coherence score")
plt.show()

In [ ]:
for notopics, cv in zip(x, coherence_values):
    print("Num Topics =", notopics, " has Coherence Value of", round(cv, 4))

In [ ]:
optimal_model = model_list[18]
for idx, topic in optimal_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
for t in range(optimal_model.num_topics):
  plt.figure(figsize = (8, 8))
  wordcloud=WordCloud(width=800,height=800,background_color="black",min_font_size=10).generate_from_frequencies(dict(optimal_model.show_topic(2, 200)))
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.show()
#optimal_model.show_topic(1, 200)

In [ ]:
subdata=[];
for i in range(0,5):
  subdata.append(Finaldata["FinalText"].sample(frac=0.8))
#for k in range(1,101,5):
for sample in subdata:
  sub_dictionary=gensim.corpora.Dictionary(sample)
  sub_bow_corpus = [sub_dictionary.doc2bow(doc) for doc in sample]
  model_list, coherence_values = compute_coherence_value(dictionary=sub_dictionary, corpus=sub_bow_corpus,texts=sample, start=1, limit=101, step=5)
  limit=101; start=1; step=5;
  x = range(start, limit, step)
  plt.figure(figsize=(12,12))
  plt.plot(x, coherence_values)
  plt.xlabel("Net number of Topics")
  plt.ylabel("Coherence score")
  plt.show()

In [ ]:
def jaccard_index(list1,list2):
  s1=set(list1);
  s2=set(list2)
  return float(len(s1.intersection(s2))) / float(len(s1.union(s2)))

In [ ]:
def hungarian method():
  

In [ ]:
def agree(S0,Sx):
  

In [ ]:
for i in range(0,51):
  subdata.append(Finaldata["FinalText"].sample(frac=0.8))
for k in range(10,20,10):
  lda_model_base =  gensim.models.LdaMulticore(bow_corpus,num_topics = 10,id2word = dictionary,passes = 10,workers = 2)
  x=lda_model_base.show_topics(num_words=10)
  S0={}
  for topic,word in x:
    S0[topic]=re.sub('[^A-Za-z ]+','',word)
  for sample in subdata:
    sub_dictionary=gensim.corpora.Dictionary(sample)
    sub_bow_corpus = [sub_dictionary.doc2bow(doc) for doc in sample]
    lda_model_sub =  gensim.models.LdaMulticore(sub_bow_corpus,num_topics = k,id2word =sub_dictionary,passes = 10,workers = 2)
    y=lda_model_base.show_topics(num_words=10)
    Sx={}
    for topic,word in x:
      Sx[topic]=re.sub('[^A-Za-z ]+','',word)
print(Sx)
print(S0)


